In [ ]:
!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AddedToken, GPT2Config, GPT2Tokenizer #, GPT2ForSequenceClassification, GPT2Model
from text_utils import custom_data_gen, DBpedia, TextProcessor, TextTokenizer
from tqdm.autonotebook import tqdm
from utils import META_PATH

In [ ]:
LR = 1e-4
LR_DECAY = 0.8
MIN_LR = 5e-6
N_EPOCHS = 1
PLATEAU = 3
BATCH_SIZE = 256
MAX_LEN = 100
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({
    "pad_token": AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)
})

In [ ]:
## to do
## re split: part of train use for quantization, part of test use for examples

## just change contents of csv

__Plan__

__1.1. re split data 19:05 DONE__
  
__1.2. new dataloaders 19:20 DONE__

__2. write train loop 19:40__

__3. train gpt2 several attention layers 20:00__

__1. re split data__

In [ ]:
pwd

In [ ]:
# data = DBpedia(root="/root/quantized_nn/dbpedia_data")

# train_data = pd.read_csv("dbpedia_data/dbpedia_csv/train.csv", header=None)
# test_data = pd.read_csv("dbpedia_data/dbpedia_csv/test.csv", header=None)

# train_data.columns = ["class", "header", "content"]
# test_data.columns = ["class", "header", "content"]



# train_data, quant_data = train_test_split(train_data,
#                                           test_size=0.2,
#                                           stratify=train_data["class"],
#                                           random_state=1234567890)
# test_data, infer_data = train_test_split(test_data,
#                                          test_size=0.5,
#                                          stratify=test_data["class"],
#                                          random_state=1234567890)

# os.mkdir("data_db/")
# train_data.to_csv("data_db/train.csv", index=None)
# quant_data.to_csv("data_db/quant.csv", index=None)
# test_data.to_csv("data_db/test.csv", index=None)
# infer_data.to_csv("data_db/infer.csv", index=None)

__2. NEW DATALOADERS__

In [ ]:
# data_dir = "data_db/"

In [ ]:
train_data = pd.read_csv("data_db/train.csv")
quant_data = pd.read_csv("data_db/quant.csv")
test_data = pd.read_csv("data_db/test.csv")
infer_data = pd.read_csv("data_db/infer.csv")
tst_0_data = train_data.iloc[:320]
tst_1_data = train_data.iloc[320:2*320]

In [ ]:
# vocab_keys = list(tokenizer.get_vocab().keys())

# train_data_1 = train_data.copy()
# train_data_2 = train_data.copy()
# train_data_3 = train_data.copy()
# train_data_4 = train_data.copy()
# train_data_5 = train_data.copy()
# train_data_6 = train_data.copy()
# train_data_7 = train_data.copy()
# train_data_8 = train_data.copy()
# train_data_9 = train_data.copy()


# def drop_random_word(x):
#     wrds = x.split()
#     idx = np.random.randint(1, max(2, len(wrds)-1))
#     return " ".join(wrds[:idx] + wrds[idx+1:])

# def get_random_word():
#     return vocab_keys[np.random.randint(0, 50256)]

# def insert_random_word(x):
#     wrds = x.split()
#     idx = np.random.randint(1, max(2, len(wrds)-1))
#     wrd = vocab_keys[np.random.randint(0, 50256)]
#     return " ".join(wrds[:idx] + [wrd] + wrds[idx:])

# def substitute_random_word(x):
#     wrds = x.split()
#     idx = np.random.randint(1, max(2, len(wrds)-1))
#     wrd = vocab_keys[np.random.randint(0, 50256)]
#     return " ".join(wrds[:idx] + [wrd] + wrds[idx+1:])

# train_data_1["content"] = [" ".join(x.split()[1:]) for x in tqdm(train_data["content"].values)]
# train_data_2["content"] = [" ".join(x.split()[:-1]) for x in tqdm(train_data["content"].values)]
# train_data_3["content"] = [" ".join(x.split()[2:]) for x in tqdm(train_data["content"].values)]
# train_data_4["content"] = [" ".join(x.split()[:-2]) for x in tqdm(train_data["content"].values)]


# np.random.seed(1234567890)
# train_data_5["content"] = [drop_random_word(x) for x in tqdm(train_data["content"].values)]
# train_data_6["content"] = [insert_random_word(x) for x in tqdm(train_data["content"].values)]
# train_data_7["content"] = [substitute_random_word(x) for x in tqdm(train_data["content"].values)]
# train_data_8["content"] = [" ".join([get_random_word()]+x.split()) for x in tqdm(train_data["content"].values)]
# train_data_9["content"] = [" ".join(x.split()+[get_random_word()]) for x in tqdm(train_data["content"].values)]



# train_data = pd.concat([
#     train_data,
#     train_data_1,
#     train_data_2,
#     train_data_3,
#     train_data_4,
#     train_data_5,
#     train_data_6,
#     train_data_7,
#     train_data_8,
#     train_data_9
# ], axis=0)

# train_data = train_data[train_data["content"] != ""]
# train_data.to_csv("data_db/train.csv", index=None)

In [ ]:
train_data.shape

In [ ]:
data_train = ((row[0], row[2]) for row in train_data.values)
data_quant = ((row[0], row[2]) for row in quant_data.values)
data_test = ((row[0], row[2]) for row in test_data.values)
data_infer = ((row[0], row[2]) for row in infer_data.values)
data_tst_0 = ((row[0], row[2]) for row in tst_0_data.values)
data_tst_1 = ((row[0], row[2]) for row in tst_1_data.values)

In [ ]:
def init_gpt2():
    gpt2_config = GPT2Config(vocab_size=len(tokenizer.get_vocab()))
    setattr(gpt2_config, "pad_token_id", 50256)
    model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path="gpt2",
                                                          config=gpt2_config)
    new_score = nn.Sequential(
        nn.Linear(in_features=768, out_features=14, bias=False),
        nn.Sigmoid()
    )

    setattr(model, "score", new_score)
    return freeze_layers(model).to(DEVICE)
    
    
def freeze_layers(model,
                  except_last=37):
    num_layers = sum(1 for p in model.parameters())
    for num, (nam, p) in enumerate(model.named_parameters()):
        if num >= num_layers - except_last:
            p.requires_grad = True
        else:
            p.requires_grad = False
    return model

In [ ]:
class Trainer:
    
    @staticmethod
    def rnd(x):
        return np.round(float(x), 5)
    
    def __init__(self,
                 model,
                 train_data,
                 test_data,
                 batch_size,
                 n_epochs,
                 plateau,
                 lr,
                 lr_decay,
                 min_lr,
                 device,
                 res_path,
                 loss_fn=None,
                 ):
        self.model = model
        self.train_data = train_data
        self.test_data = test_data
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.plateau = plateau
        self.lr = lr
        self.lr_decay = lr_decay
        self.min_lr = min_lr
        self.device = device
        self.res_path = res_path
        
        self.optimizer = optim.Adam(self.model.parameters(), self.lr)
        self.loss_fn = nn.CrossEntropyLoss()
        if loss_fn:
            self.loss_fn = getattr(nn, loss_fn)()
        self.history = {}
        self.no_gain = 0
    
    def _dump_history(self):
        with open(self.res_path, "w") as f:
            json.dump(self.history, f)
    
    def _update_optimizer(self):
        if self.lr * self.lr_decay >= self.min_lr:
            print(f"LR reduced on plateau from {self.lr} to {self.lr*self.lr_decay}")
            self.lr *= self.lr_decay
        else:
            print(f"LR will no longer be reduced: current value is {self.min_lr}")
            self.lr = self.min_lr
        self.optimizer = optim.Adam(self.model.parameters(), self.min_lr)
        self.no_gain = 0
            
    def _clear_previous_best(self):
        for file in os.listdir():
            if "gpt2_best_epoch" in file:
                os.remove(file)
    
    def _save_best_model(self,
                         epoch,
                         loss):
        self._clear_previous_best()
        filename = f"gpt2_best_epoch_{epoch}_loss_{loss}.pt"
        torch.save(self.model.state_dict(), filename)
        
    
    def _update_history(self,
                        stats):
        loss_trn, acc_trn, loss_val, acc_val = stats
        print(f"train loss - {loss_trn} , train acc - {acc_trn}")
        print(f"val.loss - {loss_val}, val.acc - {acc_val}")
        self.history["loss_trn"].append(self.rnd(loss_trn))
        self.history["acc_trn"].append(self.rnd(acc_trn))
        self.history["loss_val"].append(self.rnd(loss_val))
        self.history["acc_val"].append(self.rnd(acc_val))
        
    def _plot_history(self):
        plt.subplots(figsize=(20,6))
        plt.subplot(1,2,1)
        plt.plot(range(len(self.history["loss_trn"])), 
                           self.history["loss_trn"], 
                           color=(0,0,1), 
                           alpha=0.5,
                           label="Train.loss")
        plt.plot(range(len(self.history["loss_val"])), 
                           self.history["loss_val"], 
                           color=(1,0,0), 
                           alpha=0.5,
                           label="Val.loss")
        plt.grid()
        plt.xlim(0, self.n_epochs+1)
        plt.title("Loss dynamics")
        plt.legend(fontsize=14)
        plt.subplot(1,2,2)
        plt.plot(range(len(self.history["acc_trn"])), 
                           self.history["acc_trn"], 
                           color=(0,0,1), 
                           alpha=0.5,
                           label="Train.acc.")
        plt.plot(range(len(self.history["acc_val"])), 
                           self.history["acc_val"], 
                           color=(1,0,0), 
                           alpha=0.5,
                           label="Val.acc")
        plt.grid()
        plt.xlim(0, self.n_epochs+1)
        plt.title("Accuracy dynamics")
        plt.legend(fontsize=14)
        plt.show()
    
    def train(self):
        if os.path.exists(self.res_path):
            os.remove(self.res_path)
        self.history = {
            "loss_trn": [],
            "loss_val": [],
            "acc_trn": [],
            "acc_val": [],
            "best_loss": 1e10
        }
        for epoch in range(1, self.n_epochs+1):
            stats = self.train_epoch(epoch)
            if self.rnd(stats[2]) < self.history["best_loss"]:
                self.history["best_loss"] = self.rnd(stats[2])
                self.no_gain = 0
                self._save_best_model(epoch,
                                      self.rnd(stats[2]))
            else:
                self.no_gain += 1
            if self.no_gain == self.plateau:
                self._update_optimizer()
            self._update_history(stats)
            self._plot_history()
            self._dump_history()
        return self.model
            
    def train_epoch(self,
                    epoch):
        loss_trn = 0
        acc_trn = 0
        batch_trn = 0
        loss_val = 0
        acc_val = 0
        batch_val = 0
        self.model.to(self.device)
        self.model.train()
        data_tst_0 = ((row[0], row[2]) for row in self.train_data.values)
        data_tst_1 = ((row[0], row[2]) for row in self.test_data.values)
        print(f"Epoch {epoch}:")
        self.model.train()
        for num, (x, y) in enumerate(tqdm(custom_data_gen(data_tst_0,
                                                          tokenizer,
                                                          MAX_LEN,
                                                          self.batch_size,
                                                          self.device)),
                                     start=1):
            loss, acc = self.run_batch(x,
                                  y)
            batch_trn += 1
            loss_trn += loss.item()
            acc_trn += acc.item()
            if num % 5 == 0:
                print(f"Train loss - {self.rnd(loss_trn/num)}, train acc - {self.rnd(acc_trn/num)}", end="\r")
        
        self.model.eval()
        for num, (x, y) in enumerate(tqdm(custom_data_gen(data_tst_1,
                                                          tokenizer,
                                                          MAX_LEN,
                                                          self.batch_size,
                                                          self.device)),
                                     start=1):
            with torch.no_grad():
                loss, acc = self.run_batch(x,
                                           y,
                                           train=False)
                batch_val += 1
                loss_val += loss.item()
                acc_val += acc.item()
                if num % 5 == 0:
                    print(f"Val loss - {self.rnd(loss_val/num)}, val acc - {self.rnd(acc_val/num)}", end="\r")
        clear_output()
        return loss_trn / batch_trn, acc_trn / batch_trn, loss_val / batch_val, acc_val / batch_val
    
    def run_batch(self,
                  x,
                  y,
                  train=True):
        x = x.to(self.device)
        y = y.to(self.device)
        self.optimizer.zero_grad()
        preds = self.model(x).logits
        loss = self.loss_fn(preds, y.long())
        acc = torch.sum(torch.argmax(preds, dim=-1) == y) / self.batch_size
        if train:
            loss.backward()
            self.optimizer.step()
        return loss, acc
    

In [ ]:
model = init_gpt2()
trainer = Trainer(model,
                  train_data,
                  test_data,
                  BATCH_SIZE,
                  n_epochs=N_EPOCHS,
                  plateau=PLATEAU,
                  lr=LR,
                  lr_decay=LR_DECAY,
                  min_lr=MIN_LR,
                  device=DEVICE,
                  res_path="results_gpt2_base.json")
model_trained = trainer.train()

In [ ]:
mdl_trained = init_gpt2()

In [ ]:
checkpoint_path = [i for i in os.listdir() if "gpt2_best_epoch" in i][0]
mdl_trained.load_state_dict(torch.load(checkpoint_path))

In [ ]:
def rnd(x):
    return np.round(float(x), 5)

def run_batch(x,
              y,
              model):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        with torch.no_grad():
            preds = model(x).logits
            acc = torch.sum(torch.argmax(preds, dim=-1) == y) / BATCH_SIZE
        return acc

mdl_trained.eval()

batch_val = 0
acc_val = 0

for num, (x, y) in enumerate(tqdm(custom_data_gen(data_infer,
                                                  tokenizer,
                                                  MAX_LEN,
                                                  BATCH_SIZE,
                                                  DEVICE)),
                                     start=1):
#     with torch.no_grad():
    acc = run_batch(x,
                   y,
                    mdl_trained)
    batch_val += 1
    acc_val += acc.item()
#         if num % 5 == 0:
#             print(f"Val loss - {rnd(loss_val/num)}, val acc - {rnd(acc_val/num)}", end="\r")
total_acc = acc_val / batch_val

In [ ]:
print(total_acc)

In [ ]:
print(total_acc)

__transformers search__

1. models/gpt2/modeling_gpt2.py class GPT2ForSequenceClassification(GPT2PreTrainedModel)
2. models/gpt2/modeling_gpt2.py class GPT2PreTrainedModel(PreTrainedModel)
3. modeling_utils.py class PreTrainedModel(nn.Module, ModuleUtilsMixin, GenerationMixin, PushToHubMixin)
4. modeling_utils.py class ModuleUtilsMixin
5. generation_utils.py class GenerationMixin
6. file_utils.py class PushToHubMixin
7. models/gpt2/modeling_gpt2.py class GPT2Model(GPT2PreTrainedModel) (maybe embedding quantization can be ussed)
8. models/gpt2/modeling_gpt2.py class GPT2Block(nn.Module)
9. models/gpt2/modeling_gpt2.py class GPT2Attention(nn.Module) (__modify this to quantize attention block__)

In [ ]:
!cat /usr/local/lib/python3.6/dist-packages/transformers/models/gpt2/modeling_gpt2.py

In [ ]:
!cat /usr/local/lib/python3.6/dist-packages/transformers/activations.py

In [ ]:
from transformers.file_utils import (
    ModelOutput,
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    replace_return_docstrings,
)

from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    SequenceClassifierOutputWithPast,
)

from transformers.activations import ACT2FN

In [ ]:
_CHECKPOINT_FOR_DOC = "gpt2"
_CONFIG_FOR_DOC = "GPT2Config"
_TOKENIZER_FOR_DOC = "GPT2Tokenizer"

GPT2_PRETRAINED_MODEL_ARCHIVE_LIST = [
    "gpt2",
    "gpt2-medium",
    "gpt2-large",
    "gpt2-xl",
    "distilgpt2",
    # See all GPT-2 models at https://huggingface.co/models?filter=gpt2
]

GPT2_START_DOCSTRING = r"""

    This model inherits from :class:`~transformers.PreTrainedModel`. Check the superclass documentation for the generic
    methods the library implements for all its model (such as downloading or saving, resizing the input embeddings,
    pruning heads etc.)

    This model is also a PyTorch `torch.nn.Module <https://pytorch.org/docs/stable/nn.html#torch.nn.Module>`__
    subclass. Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to
    general usage and behavior.

    Parameters:
        config (:class:`~transformers.GPT2Config`): Model configuration class with all the parameters of the model.
            Initializing with a config file does not load the weights associated with the model, only the
            configuration. Check out the :meth:`~transformers.PreTrainedModel.from_pretrained` method to load the model
            weights.
"""

GPT2_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (:obj:`torch.LongTensor` of shape :obj:`(batch_size, input_ids_length)`):
            :obj:`input_ids_length` = ``sequence_length`` if :obj:`past_key_values` is ``None`` else
            ``past_key_values[0][0].shape[-2]`` (``sequence_length`` of input past key value states). Indices of input
            sequence tokens in the vocabulary.

            If :obj:`past_key_values` is used, only ``input_ids`` that do not have their past calculated should be
            passed as ``input_ids``.

            Indices can be obtained using :class:`~transformers.GPT2Tokenizer`. See
            :meth:`transformers.PreTrainedTokenizer.encode` and :meth:`transformers.PreTrainedTokenizer.__call__` for
            details.

            `What are input IDs? <../glossary.html#input-ids>`__
        past_key_values (:obj:`Tuple[Tuple[torch.Tensor]]` of length :obj:`config.n_layers`):
            Contains precomputed hidden-states (key and values in the attention blocks) as computed by the model (see
            :obj:`past_key_values` output below). Can be used to speed up sequential decoding. The ``input_ids`` which
            have their past given to this model should not be passed as ``input_ids`` as they have already been
            computed.
        attention_mask (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Mask to avoid performing attention on padding token indices. Mask values selected in ``[0, 1]``:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.

            `What are attention masks? <../glossary.html#attention-mask>`__
        token_type_ids (:obj:`torch.LongTensor` of shape :obj:`(batch_size, input_ids_length)`, `optional`):
            Segment token indices to indicate first and second portions of the inputs. Indices are selected in ``[0,
            1]``:

            - 0 corresponds to a `sentence A` token,
            - 1 corresponds to a `sentence B` token.

            `What are token type IDs? <../glossary.html#token-type-ids>`_
        position_ids (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Indices of positions of each input sequence tokens in the position embeddings. Selected in the range ``[0,
            config.max_position_embeddings - 1]``.

            `What are position IDs? <../glossary.html#position-ids>`_
        head_mask (:obj:`torch.FloatTensor` of shape :obj:`(num_heads,)` or :obj:`(num_layers, num_heads)`, `optional`):
            Mask to nullify selected heads of the self-attention modules. Mask values selected in ``[0, 1]``:

            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.

        inputs_embeds (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length, hidden_size)`, `optional`):
            Optionally, instead of passing :obj:`input_ids` you can choose to directly pass an embedded representation.
            This is useful if you want more control over how to convert :obj:`input_ids` indices into associated
            vectors than the model's internal embedding lookup matrix.

            If :obj:`past_key_values` is used, optionally only the last :obj:`inputs_embeds` have to be input (see
            :obj:`past_key_values`).
        use_cache (:obj:`bool`, `optional`):
            If set to :obj:`True`, :obj:`past_key_values` key value states are returned and can be used to speed up
            decoding (see :obj:`past_key_values`).
        output_attentions (:obj:`bool`, `optional`):
            Whether or not to return the attentions tensors of all attention layers. See ``attentions`` under returned
            tensors for more detail.
        output_hidden_states (:obj:`bool`, `optional`):
            Whether or not to return the hidden states of all layers. See ``hidden_states`` under returned tensors for
            more detail.
        return_dict (:obj:`bool`, `optional`):
            Whether or not to return a :class:`~transformers.file_utils.ModelOutput` instead of a plain tuple.
"""

PARALLELIZE_DOCSTRING = r"""
    This is an experimental feature and is a subject to change at a moment's notice.

    Uses a device map to distribute attention modules of the model across several devices. If no device map is given,
    it will evenly distribute blocks across all devices.

    Args:
        device_map (:obj:`Dict[int, list]`, optional, defaults to None):
            A dictionary that maps attention modules to devices. Note that the embedding module and LMHead are always
            automatically mapped to the first device (for esoteric reasons). That means that the first device should
            have fewer attention modules mapped to it than other devices. For reference, the gpt2 models have the
            following number of attention modules:

                - gpt2: 12
                - gpt2-medium: 24
                - gpt2-large: 36
                - gpt2-xl: 48

    Example::

            # Here is an example of a device map on a machine with 4 GPUs using gpt2-xl, which has a total of 48 attention modules:
            model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
            device_map = {0: [0, 1, 2, 3, 4, 5, 6, 7, 8],

                          1: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
                          2: [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34],
                          3: [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]}
            model.parallelize(device_map)
"""
DEPARALLELIZE_DOCSTRING = r"""
    Moves the model to cpu from a model parallel state.

    Example::

        # On a 4 GPU machine with gpt2-large:
        model = GPT2LMHeadModel.from_pretrained('gpt2-large')
        device_map = {0: [0, 1, 2, 3, 4, 5, 6, 7],

                    1: [8, 9, 10, 11, 12, 13, 14, 15],
                    2: [16, 17, 18, 19, 20, 21, 22, 23],
                    3: [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]}
        model.parallelize(device_map) # Splits the model across several devices
        model.deparallelize() # Put the model back on cpu and cleans memory by calling torch.cuda.empty_cache()
"""

In [ ]:
from transformers import GPT2PreTrainedModel, GPT2Model

In [ ]:
class GPT2ForSequenceClassification(GPT2PreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"h\.\d+\.attn\.masked_bias", r"lm_head\.weight"]

    def __init__(self, config, quantization=None):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.transformer = GPT2Model(config, quantization=quantization)
        self.score = nn.Linear(config.n_embd, self.num_labels, bias=False)

        self.init_weights()

        # Model parallel
        self.model_parallel = False
        self.device_map = None

    @add_start_docstrings_to_model_forward(GPT2_INPUTS_DOCSTRING)
    @add_code_sample_docstrings(
        tokenizer_class=_TOKENIZER_FOR_DOC,
        checkpoint="microsoft/DialogRPT-updown",
        output_type=SequenceClassifierOutputWithPast,
        config_class=_CONFIG_FOR_DOC,
    )
    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.transformer(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = transformer_outputs[0]
        logits = self.score(hidden_states)

        if input_ids is not None:
            batch_size, sequence_length = input_ids.shape[:2]
        else:
            batch_size, sequence_length = inputs_embeds.shape[:2]

        assert (
            self.config.pad_token_id is not None or batch_size == 1
        ), "Cannot handle batch sizes > 1 if no padding token is defined."
        if self.config.pad_token_id is None:
            sequence_lengths = -1
        else:
            if input_ids is not None:
                sequence_lengths = torch.ne(input_ids, self.config.pad_token_id).sum(-1) - 1
            else:
                sequence_lengths = -1
                logger.warning(
                    f"{self.__class__.__name__} will not detect padding tokens in `inputs_embeds`. Results may be "
                    f"unexpected if using padding tokens in conjunction with `inputs_embeds.`"
                )

        pooled_logits = logits[range(batch_size), sequence_lengths]

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(pooled_logits.view(-1), labels.to(self.dtype).view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(pooled_logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (pooled_logits,) + transformer_outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutputWithPast(
            loss=loss,
            logits=pooled_logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
        )

In [ ]:
from transformers import add_start_docstrings

In [ ]:
from quantizer import QuantDynamic

In [ ]:
_CHECKPOINT_FOR_DOC = "gpt2"
_CONFIG_FOR_DOC = "GPT2Config"
_TOKENIZER_FOR_DOC = "GPT2Tokenizer"

GPT2_PRETRAINED_MODEL_ARCHIVE_LIST = [
    "gpt2",
    "gpt2-medium",
    "gpt2-large",
    "gpt2-xl",
    "distilgpt2",
    # See all GPT-2 models at https://huggingface.co/models?filter=gpt2
]

GPT2_START_DOCSTRING = r"""

    This model inherits from :class:`~transformers.PreTrainedModel`. Check the superclass documentation for the generic
    methods the library implements for all its model (such as downloading or saving, resizing the input embeddings,
    pruning heads etc.)

    This model is also a PyTorch `torch.nn.Module <https://pytorch.org/docs/stable/nn.html#torch.nn.Module>`__
    subclass. Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to
    general usage and behavior.

    Parameters:
        config (:class:`~transformers.GPT2Config`): Model configuration class with all the parameters of the model.
            Initializing with a config file does not load the weights associated with the model, only the
            configuration. Check out the :meth:`~transformers.PreTrainedModel.from_pretrained` method to load the model
            weights.
"""

GPT2_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (:obj:`torch.LongTensor` of shape :obj:`(batch_size, input_ids_length)`):
            :obj:`input_ids_length` = ``sequence_length`` if :obj:`past_key_values` is ``None`` else
            ``past_key_values[0][0].shape[-2]`` (``sequence_length`` of input past key value states). Indices of input
            sequence tokens in the vocabulary.

            If :obj:`past_key_values` is used, only ``input_ids`` that do not have their past calculated should be
            passed as ``input_ids``.

            Indices can be obtained using :class:`~transformers.GPT2Tokenizer`. See
            :meth:`transformers.PreTrainedTokenizer.encode` and :meth:`transformers.PreTrainedTokenizer.__call__` for
            details.

            `What are input IDs? <../glossary.html#input-ids>`__
        past_key_values (:obj:`Tuple[Tuple[torch.Tensor]]` of length :obj:`config.n_layers`):
            Contains precomputed hidden-states (key and values in the attention blocks) as computed by the model (see
            :obj:`past_key_values` output below). Can be used to speed up sequential decoding. The ``input_ids`` which
            have their past given to this model should not be passed as ``input_ids`` as they have already been
            computed.
        attention_mask (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Mask to avoid performing attention on padding token indices. Mask values selected in ``[0, 1]``:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.

            `What are attention masks? <../glossary.html#attention-mask>`__
        token_type_ids (:obj:`torch.LongTensor` of shape :obj:`(batch_size, input_ids_length)`, `optional`):
            Segment token indices to indicate first and second portions of the inputs. Indices are selected in ``[0,
            1]``:

            - 0 corresponds to a `sentence A` token,
            - 1 corresponds to a `sentence B` token.

            `What are token type IDs? <../glossary.html#token-type-ids>`_
        position_ids (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Indices of positions of each input sequence tokens in the position embeddings. Selected in the range ``[0,
            config.max_position_embeddings - 1]``.

            `What are position IDs? <../glossary.html#position-ids>`_
        head_mask (:obj:`torch.FloatTensor` of shape :obj:`(num_heads,)` or :obj:`(num_layers, num_heads)`, `optional`):
            Mask to nullify selected heads of the self-attention modules. Mask values selected in ``[0, 1]``:

            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.

        inputs_embeds (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length, hidden_size)`, `optional`):
            Optionally, instead of passing :obj:`input_ids` you can choose to directly pass an embedded representation.
            This is useful if you want more control over how to convert :obj:`input_ids` indices into associated
            vectors than the model's internal embedding lookup matrix.

            If :obj:`past_key_values` is used, optionally only the last :obj:`inputs_embeds` have to be input (see
            :obj:`past_key_values`).
        use_cache (:obj:`bool`, `optional`):
            If set to :obj:`True`, :obj:`past_key_values` key value states are returned and can be used to speed up
            decoding (see :obj:`past_key_values`).
        output_attentions (:obj:`bool`, `optional`):
            Whether or not to return the attentions tensors of all attention layers. See ``attentions`` under returned
            tensors for more detail.
        output_hidden_states (:obj:`bool`, `optional`):
            Whether or not to return the hidden states of all layers. See ``hidden_states`` under returned tensors for
            more detail.
        return_dict (:obj:`bool`, `optional`):
            Whether or not to return a :class:`~transformers.file_utils.ModelOutput` instead of a plain tuple.
"""

PARALLELIZE_DOCSTRING = r"""
    This is an experimental feature and is a subject to change at a moment's notice.

    Uses a device map to distribute attention modules of the model across several devices. If no device map is given,
    it will evenly distribute blocks across all devices.

    Args:
        device_map (:obj:`Dict[int, list]`, optional, defaults to None):
            A dictionary that maps attention modules to devices. Note that the embedding module and LMHead are always
            automatically mapped to the first device (for esoteric reasons). That means that the first device should
            have fewer attention modules mapped to it than other devices. For reference, the gpt2 models have the
            following number of attention modules:

                - gpt2: 12
                - gpt2-medium: 24
                - gpt2-large: 36
                - gpt2-xl: 48

    Example::

            # Here is an example of a device map on a machine with 4 GPUs using gpt2-xl, which has a total of 48 attention modules:
            model = GPT2LMHeadModel.from_pretrained('gpt2-xl')
            device_map = {0: [0, 1, 2, 3, 4, 5, 6, 7, 8],

                          1: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
                          2: [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34],
                          3: [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]}
            model.parallelize(device_map)
"""
DEPARALLELIZE_DOCSTRING = r"""
    Moves the model to cpu from a model parallel state.

    Example::

        # On a 4 GPU machine with gpt2-large:
        model = GPT2LMHeadModel.from_pretrained('gpt2-large')
        device_map = {0: [0, 1, 2, 3, 4, 5, 6, 7],

                    1: [8, 9, 10, 11, 12, 13, 14, 15],
                    2: [16, 17, 18, 19, 20, 21, 22, 23],
                    3: [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]}
        model.parallelize(device_map) # Splits the model across several devices
        model.deparallelize() # Put the model back on cpu and cleans memory by calling torch.cuda.empty_cache()
"""

In [ ]:
class GPT2Model(GPT2PreTrainedModel):
    def __init__(self, config, quantization=None):
        super().__init__(config)

        self.embed_dim = config.hidden_size

        self.wte = nn.Embedding(config.vocab_size, self.embed_dim)
        self.wpe = nn.Embedding(config.max_position_embeddings, self.embed_dim)

        self.drop = nn.Dropout(config.embd_pdrop)
        self.h = nn.ModuleList([GPT2Block(config, quantization=quantization) for _ in range(config.num_hidden_layers)])
        self.ln_f = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_epsilon)

        self.init_weights()

        # Model parallel
        self.model_parallel = False
        self.device_map = None

    @add_start_docstrings(PARALLELIZE_DOCSTRING)
    def parallelize(self, device_map=None):
        # Check validity of device_map
        self.device_map = (
            get_device_map(len(self.h), range(torch.cuda.device_count())) if device_map is None else device_map
        )
        assert_device_map(self.device_map, len(self.h))
        self.model_parallel = True
        self.first_device = "cpu" if "cpu" in self.device_map.keys() else "cuda:" + str(min(self.device_map.keys()))
        self.last_device = "cuda:" + str(max(self.device_map.keys()))
        self.wte = self.wte.to(self.first_device)
        self.wpe = self.wpe.to(self.first_device)
        # Load onto devices
        for k, v in self.device_map.items():
            for block in v:
                cuda_device = "cuda:" + str(k)
                self.h[block] = self.h[block].to(cuda_device)
        # ln_f to last
        self.ln_f = self.ln_f.to(self.last_device)

    @add_start_docstrings(DEPARALLELIZE_DOCSTRING)
    def deparallelize(self):
        self.model_parallel = False
        self.device_map = None
        self.first_device = "cpu"
        self.last_device = "cpu"
        self.wte = self.wte.to("cpu")
        self.wpe = self.wpe.to("cpu")
        for index in range(len(self.h)):
            self.h[index] = self.h[index].to("cpu")
        self.ln_f = self.ln_f.to("cpu")
        torch.cuda.empty_cache()

    def get_input_embeddings(self):
        return self.wte

    def set_input_embeddings(self, new_embeddings):
        self.wte = new_embeddings

    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer}
        """
        for layer, heads in heads_to_prune.items():
            self.h[layer].attn.prune_heads(heads)

    @add_start_docstrings_to_model_forward(GPT2_INPUTS_DOCSTRING)
    @add_code_sample_docstrings(
        tokenizer_class=_TOKENIZER_FOR_DOC,
        checkpoint=_CHECKPOINT_FOR_DOC,
        output_type=BaseModelOutputWithPastAndCrossAttentions,
        config_class=_CONFIG_FOR_DOC,
    )
    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
            input_ids = input_ids.view(-1, input_shape[-1])
            batch_size = input_ids.shape[0]
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
            batch_size = inputs_embeds.shape[0]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        if token_type_ids is not None:
            token_type_ids = token_type_ids.view(-1, input_shape[-1])
        if position_ids is not None:
            position_ids = position_ids.view(-1, input_shape[-1])

        if past_key_values is None:
            past_length = 0
            past_key_values = tuple([None] * len(self.h))
        else:
            past_length = past_key_values[0][0].size(-2)
        if position_ids is None:
            position_ids = torch.arange(past_length, input_shape[-1] + past_length, dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0).view(-1, input_shape[-1])

        # GPT2Attention mask.
        if attention_mask is not None:
            assert batch_size > 0, "batch_size has to be defined and > 0"
            attention_mask = attention_mask.view(batch_size, -1)
            # We create a 3D attention mask from a 2D tensor mask.
            # Sizes are [batch_size, 1, 1, to_seq_length]
            # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
            # this attention mask is more simple than the triangular masking of causal attention
            # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
            attention_mask = attention_mask[:, None, None, :]

            # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
            # masked positions, this operation will create a tensor which is 0.0 for
            # positions we want to attend and -10000.0 for masked positions.
            # Since we are adding it to the raw scores before the softmax, this is
            # effectively the same as removing these entirely.
            attention_mask = attention_mask.to(dtype=self.dtype)  # fp16 compatibility
            attention_mask = (1.0 - attention_mask) * -10000.0

        # If a 2D ou 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.add_cross_attention and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # head_mask has shape n_layer x batch x n_heads x N x N
        head_mask = self.get_head_mask(head_mask, self.config.n_layer)

        if inputs_embeds is None:
            inputs_embeds = self.wte(input_ids)
        position_embeds = self.wpe(position_ids)
        hidden_states = inputs_embeds + position_embeds

        if token_type_ids is not None:
            token_type_embeds = self.wte(token_type_ids)
            hidden_states = hidden_states + token_type_embeds

        hidden_states = self.drop(hidden_states)

        output_shape = input_shape + (hidden_states.size(-1),)

        presents = () if use_cache else None
        all_self_attentions = () if output_attentions else None
        all_cross_attentions = () if output_attentions and self.config.add_cross_attention else None
        all_hidden_states = () if output_hidden_states else None
        for i, (block, layer_past) in enumerate(zip(self.h, past_key_values)):

            # Model parallel
            if self.model_parallel:
                torch.cuda.set_device(hidden_states.device)
                # Ensure layer_past is on same device as hidden_states (might not be correct)
                if layer_past is not None:
                    layer_past = tuple(past_state.to(hidden_states.device) for past_state in layer_past)
                # Ensure that attention_mask is always on the same device as hidden_states
                if attention_mask is not None:
                    attention_mask = attention_mask.to(hidden_states.device)
                if isinstance(head_mask, torch.Tensor):
                    head_mask = head_mask.to(hidden_states.device)
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            if getattr(self.config, "gradient_checkpointing", False) and self.training:

                if use_cache:
                    logger.warning(
                        "`use_cache=True` is incompatible with `config.gradient_checkpointing=True`. Setting "
                        "`use_cache=False`..."
                    )
                    use_cache = False

                def create_custom_forward(module):
                    def custom_forward(*inputs):
                        # None for past_key_value
                        return module(*inputs, use_cache, output_attentions)

                    return custom_forward

                outputs = torch.utils.checkpoint.checkpoint(
                    create_custom_forward(block),
                    hidden_states,
                    None,
                    attention_mask,
                    head_mask[i],
                    encoder_hidden_states,
                    encoder_attention_mask,
                )
            else:
                outputs = block(
                    hidden_states,
                    layer_past=layer_past,
                    attention_mask=attention_mask,
                    head_mask=head_mask[i],
                    encoder_hidden_states=encoder_hidden_states,
                    encoder_attention_mask=encoder_attention_mask,
                    use_cache=use_cache,
                    output_attentions=output_attentions,
                )

            hidden_states = outputs[0]
            if use_cache is True:
                presents = presents + (outputs[1],)

            if output_attentions:
                all_self_attentions = all_self_attentions + (outputs[2 if use_cache else 1],)
                if self.config.add_cross_attention:
                    all_cross_attentions = all_cross_attentions + (outputs[3 if use_cache else 2],)

            # Model Parallel: If it's the last layer for that device, put things on the next device
            if self.model_parallel:
                for k, v in self.device_map.items():
                    if i == v[-1] and "cuda:" + str(k) != self.last_device:
                        hidden_states = hidden_states.to("cuda:" + str(k + 1))

        hidden_states = self.ln_f(hidden_states)

        hidden_states = hidden_states.view(*output_shape)
        # Add last hidden state
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(v for v in [hidden_states, presents, all_hidden_states, all_self_attentions] if v is not None)

        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,
            past_key_values=presents,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
            cross_attentions=all_cross_attentions,
        )

In [ ]:
class GPT2Block(nn.Module):
    def __init__(self, config, quantization=None):
        super().__init__()
        hidden_size = config.hidden_size
        inner_dim = config.n_inner if config.n_inner is not None else 4 * hidden_size
        
        self.attn = GPT2Attention(config, quantization=quantization)
        self.ln_1 = nn.LayerNorm(hidden_size, eps=config.layer_norm_epsilon)
        self.ln_2 = nn.LayerNorm(hidden_size, eps=config.layer_norm_epsilon)
        

        if config.add_cross_attention:
            self.crossattention = GPT2Attention(config, is_cross_attention=True, quantization=quantization)
            self.ln_cross_attn = nn.LayerNorm(hidden_size, eps=config.layer_norm_epsilon)

        self.mlp = GPT2MLP(inner_dim, config)

    def forward(
        self,
        hidden_states,
        layer_past=None,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        residual = hidden_states
        hidden_states = self.ln_1(hidden_states)
        attn_outputs = self.attn(
            hidden_states,
            layer_past=layer_past,
            attention_mask=attention_mask,
            head_mask=head_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
        )
        attn_output = attn_outputs[0]  # output_attn: a, present, (attentions)
        outputs = attn_outputs[1:]
        # residual connection
        hidden_states = attn_output + residual

        if encoder_hidden_states is not None:
            # add one self-attention block for cross-attention
            if not hasattr(self, "crossattention"):
                raise ValueError(
                    f"If `encoder_hidden_states` are passed, {self} has to be instantiated with "
                    "cross-attention layers by setting `config.add_cross_attention=True`"
                )
            residual = hidden_states
            hidden_states = self.ln_cross_attn(hidden_states)
            cross_attn_outputs = self.crossattention(
                hidden_states,
                attention_mask=attention_mask,
                head_mask=head_mask,
                encoder_hidden_states=encoder_hidden_states,
                encoder_attention_mask=encoder_attention_mask,
                output_attentions=output_attentions,
            )
            attn_output = cross_attn_outputs[0]
            # residual connection
            hidden_states = residual + attn_output
            outputs = outputs + cross_attn_outputs[2:]  # add cross attentions if we output attention weights

        residual = hidden_states
        hidden_states = self.ln_2(hidden_states)
        feed_forward_hidden_states = self.mlp(hidden_states)
        # residual connection
        hidden_states = residual + feed_forward_hidden_states

        if use_cache:
            outputs = (hidden_states,) + outputs
        else:
            outputs = (hidden_states,) + outputs[1:]

        return outputs  # hidden_states, present, (attentions, cross_attentions)
    

class GPT2Attention(nn.Module):
    def __init__(self, config, is_cross_attention=False, quantization=None):
        super().__init__()

        max_positions = config.max_position_embeddings
        self.register_buffer(
            "bias",
            torch.tril(torch.ones((max_positions, max_positions), dtype=torch.uint8)).view(
                1, 1, max_positions, max_positions
            ),
        )
        self.register_buffer("masked_bias", torch.tensor(-1e4))

        self.embed_dim = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.embed_dim // self.num_heads
        self.split_size = self.embed_dim
        if self.head_dim * self.num_heads != self.embed_dim:
            raise ValueError(
                f"`embed_dim` must be divisible by num_heads (got `embed_dim`: {self.embed_dim} and `num_heads`: {self.num_heads})."
            )

        self.scale_attn_weights = config.scale_attn_weights
        self.is_cross_attention = is_cross_attention
        self.quantization = quantization

        if self.is_cross_attention:
            self.c_attn = Conv1D(2 * self.embed_dim, self.embed_dim, self.quantization)
            self.q_attn = Conv1D(self.embed_dim, self.embed_dim, self.quantization)
        else:
            self.c_attn = Conv1D(3 * self.embed_dim, self.embed_dim, self.quantization)
        self.c_proj = Conv1D(self.embed_dim, self.embed_dim, self.quantization)

        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)

        self.pruned_heads = set()

    def prune_heads(self, heads):
        if len(heads) == 0:
            return
        heads, index = find_pruneable_heads_and_indices(heads, self.num_heads, self.head_dim, self.pruned_heads)
        index_attn = torch.cat([index, index + self.split_size, index + (2 * self.split_size)])

        # Prune conv1d layers
        self.c_attn = prune_conv1d_layer(self.c_attn, index_attn, dim=1)
        self.c_proj = prune_conv1d_layer(self.c_proj, index, dim=0)

        # Update hyper params
        self.split_size = (self.split_size // self.num_heads) * (self.num_heads - len(heads))
        self.num_heads = self.num_heads - len(heads)
        self.pruned_heads = self.pruned_heads.union(heads)

    def _attn(self, query, key, value, attention_mask=None, head_mask=None):
        attn_weights = torch.matmul(query, key.transpose(-1, -2))

        if self.scale_attn_weights:
            attn_weights = attn_weights / (float(value.size(-1)) ** 0.5)

        if not self.is_cross_attention:
            # if only "normal" attention layer implements causal mask
            query_length, key_length = query.size(-2), key.size(-2)
            causal_mask = self.bias[:, :, key_length - query_length : key_length, :key_length].bool()
            attn_weights = torch.where(causal_mask, attn_weights, self.masked_bias.to(attn_weights.dtype))

        if attention_mask is not None:
            # Apply the attention mask
            attn_weights = attn_weights + attention_mask

        attn_weights = nn.Softmax(dim=-1)(attn_weights)
        attn_weights = self.attn_dropout(attn_weights)

        # Mask heads if we want to
        if head_mask is not None:
            attn_weights = attn_weights * head_mask

        attn_output = torch.matmul(attn_weights, value)

        return attn_output, attn_weights

    def _split_heads(self, tensor, num_heads, attn_head_size):
        """
        Splits hidden_size dim into attn_head_size and num_heads
        """
        new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
        tensor = tensor.view(*new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

    def _merge_heads(self, tensor, num_heads, attn_head_size):
        """
        Merges attn_head_size dim and num_attn_heads dim into hidden_size
        """
        tensor = tensor.permute(0, 2, 1, 3).contiguous()
        new_shape = tensor.size()[:-2] + (num_heads * attn_head_size,)
        return tensor.view(new_shape)

    def forward(
        self,
        hidden_states,
        layer_past=None,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        if encoder_hidden_states is not None:
            if not hasattr(self, "q_attn"):
                raise ValueError(
                    "If class is used as cross attention, the weights `q_attn` have to be defined. "
                    "Please make sure to instantiate class with `GPT2Attention(..., is_cross_attention=True)`."
                )

            query = self.q_attn(hidden_states)
            key, value = self.c_attn(encoder_hidden_states).split(self.split_size, dim=2)
            attention_mask = encoder_attention_mask
        else:
            query, key, value = self.c_attn(hidden_states).split(self.split_size, dim=2)

        query = self._split_heads(query, self.num_heads, self.head_dim)
        key = self._split_heads(key, self.num_heads, self.head_dim)
        value = self._split_heads(value, self.num_heads, self.head_dim)

        if layer_past is not None:
            past_key, past_value = layer_past
            key = torch.cat((past_key, key), dim=-2)
            value = torch.cat((past_value, value), dim=-2)

        if use_cache is True:
            present = (key, value)
        else:
            present = None

        attn_output, attn_weights = self._attn(query, key, value, attention_mask, head_mask)

        attn_output = self._merge_heads(attn_output, self.num_heads, self.head_dim)
        attn_output = self.c_proj(attn_output)
        attn_output = self.resid_dropout(attn_output)

        outputs = (attn_output, present)
        if output_attentions:
            outputs += (attn_weights,)

        return outputs  # a, present, (attentions)


class GPT2MLP(nn.Module):
    def __init__(self, intermediate_size, config, quantization=None):
        super().__init__()
        embed_dim = config.hidden_size
        self.c_fc = Conv1D(intermediate_size, embed_dim, quantization=quantization)
        self.c_proj = Conv1D(embed_dim, intermediate_size, quantization=quantization)
        self.act = ACT2FN[config.activation_function]
        self.dropout = nn.Dropout(config.resid_pdrop)

    def forward(self, hidden_states):
        hidden_states = self.c_fc(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.c_proj(hidden_states)
        hidden_states = self.dropout(hidden_states)
        return hidden_states


class Conv1D(nn.Module):
    """
    1D-convolutional layer as defined by Radford et al. for OpenAI GPT (and also used in GPT-2).

    Basically works like a linear layer but the weights are transposed.

    Args:
        nf (:obj:`int`): The number of output features.
        nx (:obj:`int`): The number of input features.
    """

    def __init__(self, nf, nx, quantization=None):
        super().__init__()
        self.nf = nf
        w = torch.empty(nx, nf)
        nn.init.normal_(w, std=0.02)
        self.weight = nn.Parameter(w)
        self.bias = nn.Parameter(torch.zeros(nf))
        self.quantization = quantization
        
        # parameters for static quantization
        self.min_val = 0
        self.max_val = 0
        
    def 

    def forward(self, x):
        size_out = x.size()[:-1] + (self.nf,)
        if self.quantization:
            quant_wts = quantize_dynamic(self.weight, self.quantization)
            quant_x = quantize_dynamic(x, self.quantization)
            x = torch.addmm(self.bias, quant_x.view(-1, quant_x.size(-1)), quant_wts)
            x = x.view(*size_out)
            return x
        else:
            x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
            x = x.view(*size_out)
            return x
        

class LayerNorm(nn.LayerNorm):
    
    def __init__(self, normalized_shape: int, eps: float = 1e-5, quantization=None, elementwise_affine: bool = True):
        super(LayerNorm, self).__init__(normalized_shape, eps)
        self.quantization = quantization
        
    def forward(self, x):
        if self.quantization:
            quant_wts = quantize_dynamic(self.weight, self.quantization)
            quant_x = quantize_dynamic(x, self.quantization)
            return F.layer_norm(
                quant_x, self.normalized_shape, quant_wts, self.bias, self.eps)
        else:
            return F.layer_norm(
                    x, self.normalized_shape, self.weight, self.bias, self.eps)
        


In [1]:
class TestClass:
    
    def __init__(self):
        self.min_val = None
        
    def return_min_val(self,
                       x):
        if not self.min_val:
            self.min_val = x
        return self.min_val

In [ ]:
def init_gpt2(quantization=None):
    gpt2_config = GPT2Config(vocab_size=len(tokenizer.get_vocab()))
    setattr(gpt2_config, "pad_token_id", 50256)
    model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path="gpt2",
                                                          config=gpt2_config,
                                                          quantization=quantization)
    new_score = nn.Sequential(
        nn.Linear(in_features=768, out_features=14, bias=False),
        nn.Sigmoid()
    )

    setattr(model, "score", new_score)
    return freeze_layers(model).to(DEVICE)
    
    
def freeze_layers(model,
                  except_last=37):
    num_layers = sum(1 for p in model.parameters())
    for num, (nam, p) in enumerate(model.named_parameters()):
        if num >= num_layers - except_last:
            p.requires_grad = True
        else:
            p.requires_grad = False
    return model


def init_data():
    data_train = ((row[0], row[2]) for row in train_data.values)
    data_quant = ((row[0], row[2]) for row in quant_data.values)
    data_test = ((row[0], row[2]) for row in test_data.values)
    data_infer = ((row[0], row[2]) for row in infer_data.values)
    data_tst_0 = ((row[0], row[2]) for row in tst_0_data.values)
    data_tst_1 = ((row[0], row[2]) for row in tst_1_data.values)
    return data_train, data_quant, data_test, data_infer, data_tst_0, data_tst_1



In [ ]:
nn.LayerNorm?

In [ ]:
QUANTIZATION = None

mdl_trained = init_gpt2(QUANTIZATION)
data_train, data_quant, data_test, data_infer, data_tst_0, data_tst_1 = init_data()

checkpoint_path = [i for i in os.listdir() if "gpt2_best_epoch" in i][0]
mdl_trained.load_state_dict(torch.load(checkpoint_path))

def rnd(x):
    return np.round(float(x), 5)

def run_batch(x,
              y,
              model):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        with torch.no_grad():
            preds = model(x).logits
            acc = torch.sum(torch.argmax(preds, dim=-1) == y) / BATCH_SIZE
        return acc

mdl_trained.eval()

batch_val = 0
acc_val = 0

for num, (x, y) in enumerate(tqdm(custom_data_gen(data_infer,
                                                  tokenizer,
                                                  MAX_LEN,
                                                  BATCH_SIZE,
                                                  DEVICE)),
                                     start=1):
#     with torch.no_grad():
    acc = run_batch(x,
                   y,
                    mdl_trained)
    batch_val += 1
    acc_val += acc.item()
#         if num % 5 == 0:
#             print(f"Val loss - {rnd(loss_val/num)}, val acc - {rnd(acc_val/num)}", end="\r")
total_acc = acc_val / batch_val

print(total_acc)

In [ ]:
# Code adapted from: https://github.com/jafermarq/WinogradAwareNets/blob/master/src/quantization.py

def quantize_dynamic(input, num_bits=8, stochastic=False, out_half=False):
    
    output = input.clone()
    
    min_val = input.detach().min()
    max_val = input.detach().max()
    
    qmin = -1.0 * (2**num_bits)/2
    qmax = -qmin - 1

    # compute qparams --> scale and zero_point
    max_val, min_val = float(max_val), float(min_val)
    min_val = min(0.0, min_val)
    max_val = max(0.0, max_val)

    if max_val == min_val:
        scale = 1.0
        zero_point = 0
    else:
        max_range = max(-min_val, max_val) # largest mag(value)
        scale = max_range / ((qmax - qmin) / 2)
        scale = max(scale, 1e-8)
        zero_point = 0.0 # this true for symmetric quantization

    if stochastic:
        noise = output.new(output.shape).uniform_(-0.5, 0.5)
        output.add_(noise)

    output.div_(scale).add_(zero_point)
    output.round_().clamp_(qmin, qmax)  # quantize
    output.add_(-zero_point).mul_(scale)  # dequantize

    if out_half and num_bits <= 16:
        output = output.half()

    return output

#     @staticmethod
#     def backward(ctx, grad_output):
#         # straight-through estimator
#         grad_input = grad_output
#         return grad_input, None, None, None


# class QuantDynamic(nn.Module):

#     def __init__(self, num_bits=8, warmup: bool = False):
#         super(QuantDynamic, self).__init__()
#         self.register_buffer('min_val', torch.zeros(1))
#         self.register_buffer('max_val', torch.zeros(1))
#         self.num_bits = num_bits
#         self.warmup = warmup

#     def forward(self, input, isInit: bool = False):
#         """ isInit is used to quantize the --static transforms during model initialization, which is done in .eval() mode """
#         # If loading from a pretrained model with normal convolutions, some of the quantization layers (e..g winograd specific) won't be present in the pre-trained model. Here we give the option of using --warmup to use the max/min of the input tensor to compute the quantization ranges. if this option is not enabled, min_val == max_val, which will make scale=1 and zero_point=0 (just as PyTorch observers do)
#                 # equivalent to --> min_val = min_val(1-self.momentum) + self.momentum * torch.min(input)
#         self.min_val = input.detach().min()
#         self.max_val = input.detach().max()

#         return Quantize().apply(input, self.num_bits, self.min_val, self.max_val)

In [ ]:
QUANTIZATION = 16

mdl_trained_dynamic_16 = init_gpt2(QUANTIZATION)
data_train, data_quant, data_test, data_infer, data_tst_0, data_tst_1 = init_data()

checkpoint_path = [i for i in os.listdir() if "gpt2_best_epoch" in i][0]
mdl_trained_dynamic_16.load_state_dict(torch.load(checkpoint_path))

def rnd(x):
    return np.round(float(x), 5)

def run_batch(x,
              y,
              model):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        with torch.no_grad():
            preds = model(x).logits
            acc = torch.sum(torch.argmax(preds, dim=-1) == y) / BATCH_SIZE
        return acc

mdl_trained_dynamic_16.eval()

batch_val = 0
acc_val = 0

for num, (x, y) in enumerate(tqdm(custom_data_gen(data_infer,
                                                  tokenizer,
                                                  MAX_LEN,
                                                  BATCH_SIZE,
                                                  DEVICE)),
                                     start=1):
#     with torch.no_grad():
    acc = run_batch(x,
                    y,
                    mdl_trained_dynamic_16)
    batch_val += 1
    acc_val += acc.item()
#         if num % 5 == 0:
#             print(f"Val loss - {rnd(loss_val/num)}, val acc - {rnd(acc_val/num)}", end="\r")
total_acc = acc_val / batch_val

print(total_acc)

path_wts_dynamic_16 = "gpt2_dynamic_16.pt"
if not os.path.exists(path_wts_dynamic_16):
    torch.save(mdl_trained_dynamic_16.state_dict(), path_wts_dynamic_16)
else:
    print("path exists")

In [ ]:
QUANTIZATION = 8

mdl_trained_dynamic_8 = init_gpt2(QUANTIZATION)
data_train, data_quant, data_test, data_infer, data_tst_0, data_tst_1 = init_data()

checkpoint_path = [i for i in os.listdir() if "gpt2_best_epoch" in i][0]
mdl_trained_dynamic_8.load_state_dict(torch.load(checkpoint_path))

def rnd(x):
    return np.round(float(x), 5)

def run_batch(x,
              y,
              model):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        with torch.no_grad():
            preds = model(x).logits
            acc = torch.sum(torch.argmax(preds, dim=-1) == y) / BATCH_SIZE
        return acc

mdl_trained_dynamic_8.eval()

batch_val = 0
acc_val = 0

for num, (x, y) in enumerate(tqdm(custom_data_gen(data_infer,
                                                  tokenizer,
                                                  MAX_LEN,
                                                  BATCH_SIZE,
                                                  DEVICE)),
                                     start=1):
#     with torch.no_grad():
    acc = run_batch(x,
                    y,
                    mdl_trained_dynamic_8)
    batch_val += 1
    acc_val += acc.item()
#         if num % 5 == 0:
#             print(f"Val loss - {rnd(loss_val/num)}, val acc - {rnd(acc_val/num)}", end="\r")
total_acc = acc_val / batch_val

print(total_acc)

path_wts_dynamic_8 = "gpt2_dynamic_8.pt"
if not os.path.exists(path_wts_dynamic_8):
    torch.save(mdl_trained_dynamic_8.state_dict(), path_wts_dynamic_8)
else:
    print("path exists")

In [ ]:
QUANTIZATION = 4

mdl_trained_dynamic_4 = init_gpt2(QUANTIZATION)
data_train, data_quant, data_test, data_infer, data_tst_0, data_tst_1 = init_data()

checkpoint_path = [i for i in os.listdir() if "gpt2_best_epoch" in i][0]
mdl_trained_dynamic_4.load_state_dict(torch.load(checkpoint_path))

def rnd(x):
    return np.round(float(x), 5)

def run_batch(x,
              y,
              model):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        with torch.no_grad():
            preds = model(x).logits
            acc = torch.sum(torch.argmax(preds, dim=-1) == y) / BATCH_SIZE
        return acc

mdl_trained_dynamic_4.eval()

batch_val = 0
acc_val = 0

for num, (x, y) in enumerate(tqdm(custom_data_gen(data_infer,
                                                  tokenizer,
                                                  MAX_LEN,
                                                  BATCH_SIZE,
                                                  DEVICE)),
                                     start=1):
#     with torch.no_grad():
    acc = run_batch(x,
                    y,
                    mdl_trained_dynamic_4)
    batch_val += 1
    acc_val += acc.item()
#         if num % 5 == 0:
#             print(f"Val loss - {rnd(loss_val/num)}, val acc - {rnd(acc_val/num)}", end="\r")
total_acc = acc_val / batch_val

print(total_acc)

path_wts_dynamic_4 = "gpt2_dynamic_4.pt"
if not os.path.exists(path_wts_dynamic_4):
    torch.save(mdl_trained_dynamic_4.state_dict(), path_wts_dynamic_4)
else:
    print("path exists")

In [ ]:
inf_dynamic_8 = []

for _ in tqdm(range(100)):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    start.record()
    mdl_trained_dynamic_8(x)
    end.record()

    # Waits for everything to finish running
    torch.cuda.synchronize()
    inf_dynamic_8.append(start.elapsed_time(end))

In [ ]:
np.mean(inf_dynamic_8)

In [ ]:
inf_dynamic_16 = []

for _ in tqdm(range(100)):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    start.record()
    mdl_trained_dynamic_16(x)
    end.record()

    # Waits for everything to finish running
    torch.cuda.synchronize()
    inf_dynamic_16.append(start.elapsed_time(end))
    

In [ ]:
np.mean(inf_dynamic_16)

In [ ]:
inf_dynamic_4 = []

for _ in tqdm(range(100)):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    start.record()
    mdl_trained_dynamic_4(x)
    end.record()

    # Waits for everything to finish running
    torch.cuda.synchronize()
    inf_dynamic_4.append(start.elapsed_time(end))
    

In [ ]:
np.mean(inf_dynamic_4)

In [ ]:
for num, (n,p) in enumerate(mdl_trained.named_parameters()):
    if "ln" in n:
        print(num, n, p.size())

In [ ]:
mdl_trained.transformer.h[10].ln_1

In [ ]:
x=2
print(x)

In [ ]:
F.conv2d??

In [ ]:
all_params = sum(torch.mul(*p.view(1,-1).size()) for p in model.parameters() if len(p.size()) > 1).item()

In [ ]:
all_params

In [ ]:
all_params

In [ ]:
768*768

In [ ]:
torch.mul(*torch.Tensor([768, 768]))

In [ ]:
x = p.size()

In [ ]:
torch.mul(*x).item()